# Agenda: Week 5 (Modules and packages)

1. Recap and Q&A
2. Intro to modules
3. What do modules contain?
4. The different forms of `import`
5. Developing a module
6. Python's standard library
7. Modules vs. packages
8. PyPI and third-party modules
9. Installing things with `pip`
10. What's next?

# Local and global variables

Remember that outside of a function definition, all variables are *global*. 

Inside of a function:

- If we *set* a variable, then that variable is considered *local*.
- If we ask for a variable value, then Python:
    - First looks inside of the function for a local variable of that name
    - If it doesn't find a local variable of that name, then it looks for a global of that name
    
LEGB -- local, enclosing, global, and builtin -- is the way that Python searches for variables.

If I'm inside of a function, we might very well encounter a situation where there is a local variable with the same name as a global variable. However, based on what we've seen so far, there is no way for us to assign to the global variable of that same name. Once there's a local variable "shadowing" the global, we're out of luck.

There are two ways to get around this:

1. Use the `global` declaration in the function. This gets rid of the local variable; all references to that name are now global.
2. Import the `__main__` module from the Python standard library. That gives you access to all of the global variables via that namespace/module.

In [1]:
# example 1, using nothing -- default, problematic situation

x = 100

def myfunc():
    x = 200
    print(f'In myfunc, the value of x is {x}')
    
print(f'Before, x = {x}')    
myfunc()
print(f'After, x = {x}')

Before, x = 100
In myfunc, the value of x is 200
After, x = 100


In [2]:
# example 2, using global -- this means that assigning to x on line 6 does *NOT* create
# a local variable. All assignments to x then go to the global

# please *NEVER* use "global" in your programs unless you are absolutely, positively desperate

x = 100

def myfunc():
    global x    # this tells Python not to create a local variable for the function
    x = 200
    print(f'In myfunc, the value of x is {x}')
    
print(f'Before, x = {x}')    
myfunc()
print(f'After, x = {x}')

Before, x = 100
In myfunc, the value of x is 200
After, x = 200


In [4]:
# example 3, using __main__
# if you need to assign to a global variable, this is my preferred way of doing it

import __main__
x = 100

def myfunc():
    x = 300            # assigns to the local variable
    __main__.x = 200   # assigns to the global variable via the __main__ namespace
    print(f'In myfunc, the value of x is {x}')   # this retrieves the local value
    
print(f'Before, x = {x}')    
myfunc()
print(f'After, x = {x}')

Before, x = 100
In myfunc, the value of x is 300
After, x = 200


# Modules -- what are they good for?

We've talked about the "DRY" (don't repeat yourself) rule in programming.

It cleans up our code, making it easier to (a) write, (b) maintain, and (c) think about.

1. If you have the same code several lines in a row, then you should replace that code with a loop.
2. If you have the same code in several different parts of your program, you should write a function and then invoke the function in all of those places.
3. If you have the same code in several different programs, I can write the functionality once, and refer to it whenever I need it. This is known in the programming world as a "library," and in Python, libraries are implemented using "modules."

But modules do more than that in Python: They also provide us with *namespaces*, ensuring that we don't have "namespace collisions" -- when two or more parts of a program use the same variable name, and then end up assigning their own values to the other part's variables.

The way that modules handle this is by treating their variables as "attributes," names that come after `.`. If I collaborate with someone else on a Python program, then if I write my things in a module, and they write their things in a module, we cannot have a namespace collision, because the names will be in separate modules!

In [5]:
# we've already used a module -- let's see how we can use it!

# we use "import" to load a module
# import:
# (a) is not a function! Don't use parentheses!
# (b) the argument that we give it is not a string! 
# (c) the argument is not a filename! 

# the argument to import is actually the name of the module variable we want to define

# import 
# (a) creates a module object and
# (b) assigns that module object to a variable

import random

In [6]:
# what is random?
type(random)

module

# What does a module contain?

Python code, most typically:

- Function definitions
- Data definitions
- Class definitions, for new types of data we want to work with

This means that when I say `import random`, I have access to all of the functions that the author wrote for the `random` module. Over time, if they add new functionality there, I'll benefit; all I have to do is say `import random`.

In [8]:
# if I want to use functionality from random, it'll all be as attributes under "random."

# get a random integer from 0-100

# we ask Python to go into the "random" namespace
# execute the function "randint" in that namespace, passing it (0, 100) as arguments.
random.randint(0, 100)

19

In [9]:
# in Jupyter, we can see the definition of a function by putting ?? after its name
random.randint??

# Exercise: Guessing game

1. Choose a random integer (using `random.randint` from 0-100).
2. Ask the user to enter a guess.
3. Print whether the guess is right, too high, or too low.
    - If the user guessed correctly, then exit the program
    - Otherwise, have them try again.
    
Example:

    Guess a number: 50
    Too low!
    Guess a number: 90
    Too high!
    Guess a number: 80
    Too low!
    Guess a number: 86
    You got it!
    

In [11]:
import random    # I must import the module

number = random.randint(0, 100)    # choose a random number, and put it in "number"

while True:
    guess = input('Guess: ').strip()
    
    if not guess.isdigit():
        print(f'Not numeric! Try again!')
        continue
        
    n = int(guess)
    
    if n == number:
        print('You got it!')
        break
        
    elif n < number:
        print('Too low!')
        
    else:
        print('Too high!')

Guess: 50
Too high!
Guess: asdfafa
Not numeric! Try again!
Guess: 25
Too high!
Guess: 12
Too high!
Guess: 6
Too high!
Guess: 3
You got it!


# What else does `random` contain?

`random` is a module, which makes it a container for other names (and thus definitions). How can we find out what else it contains?

1. We use the builtin `dir` function on the `random` module, and get a list of names available in `random`.

In [12]:
# use dir

dir(random)

['BPF',
 'LOG4',
 'NV_MAGICCONST',
 'RECIP_BPF',
 'Random',
 'SG_MAGICCONST',
 'SystemRandom',
 'TWOPI',
 '_ONE',
 '_Sequence',
 '_Set',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_accumulate',
 '_acos',
 '_bisect',
 '_ceil',
 '_cos',
 '_e',
 '_exp',
 '_floor',
 '_index',
 '_inst',
 '_isfinite',
 '_log',
 '_os',
 '_pi',
 '_random',
 '_repeat',
 '_sha512',
 '_sin',
 '_sqrt',
 '_test',
 '_test_generator',
 '_urandom',
 '_warn',
 'betavariate',
 'choice',
 'choices',
 'expovariate',
 'gammavariate',
 'gauss',
 'getrandbits',
 'getstate',
 'lognormvariate',
 'normalvariate',
 'paretovariate',
 'randbytes',
 'randint',
 'random',
 'randrange',
 'sample',
 'seed',
 'setstate',
 'shuffle',
 'triangular',
 'uniform',
 'vonmisesvariate',
 'weibullvariate']